<font size="10" color="purple"><left><b><i><u>Exploratory Data Analysis #4</u></i></b></left></font>

<font size="4"><li><left><i>Instanciamos clase EDA con los métodos necesarios</i></left></font>


In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import seaborn as sb
import pandas_profiling as pdpf
from IPython.display import HTML
import sweetviz as sv

class EDA:
    def read_csv(self, file_path):
        """
        Lee un archivo CSV y devuelve un dataframe.

        Args:
            file_path (str): Ruta al archivo CSV a leer.

        Returns:
            pd.DataFrame: El dataframe con los datos del archivo CSV.
        """
        dataframe = pd.read_csv(file_path,encoding='UTF-8')
        return dataframe

    def get_columnas(self, dataframe):
        """
        Obtiene una lista de las columnas de un dataframe.

        Args:
            dataframe (pd.DataFrame): El dataframe del cual se desean obtener las columnas.

        Returns:
            list: Lista de las columnas del dataframe.
        """
        columnas = dataframe.columns.tolist()
        return columnas

    def set_columns_dtype_category(self, dataframe, columnas):
        """
        Asigna el tipo de dato "category" a una lista de columnas en un dataframe.

        Args:
            dataframe (pd.DataFrame): El dataframe al que se desea asignar los tipos de dato.
            columnas (list): La lista de columnas a las que se desea asignar el tipo de dato "category".
        """
        dataframe[columnas] = dataframe[columnas].astype('category')

    def set_columns_dtype_int(self, dataframe, columnas):
        """
        Asigna el tipo de dato "int" a una lista de columnas en un dataframe.
        Los valores nulos en las columnas seleccionadas se reemplazan por 0.

        Args:
            dataframe (pd.DataFrame): El dataframe al que se desea asignar los tipos de dato.
            columnas (list): La lista de columnas a las que se desea asignar el tipo de dato "int".
        """
        dataframe[columnas] = dataframe[columnas].astype(int).fillna(0)

    def set_columns_dtype_float(self, dataframe, columnas):
        """
        Asigna el tipo de dato "float" a una lista de columnas en un dataframe.

        Args:
            dataframe (pd.DataFrame): El dataframe al que se desea asignar los tipos de dato.
            columnas (list): La lista de columnas a las que se desea asignar el tipo de dato "float".
        """
        dataframe[columnas] = dataframe[columnas].astype(float)

    def set_columns_dtype_datetime(self, dataframe, columnas, formato='%Y-%m-%d'):
        """
        Asigna el tipo de dato "datetime" a una lista de columnas en un dataframe, con el formato especificado.

        Args:
            dataframe (pd.DataFrame): El dataframe al que se desea asignar los tipos de dato.
            columnas (list): La lista de columnas a las que se desea asignar el tipo de dato "datetime".
            formato (str): El formato de fecha a utilizar (por defecto: "%Y-%m-%d").
        """
        dataframe[columnas] = pd.to_datetime(dataframe[columnas], format=formato)

    def rename_columns(self, dataframe, column_mapping):
        """
        Modifica los nombres de las columnas de un dataframe.

        Args:
            dataframe (pd.DataFrame): El dataframe al que se le van a modificar los nombres de las columnas.
            column_mapping (dict): Un diccionario que contiene el mapeo de los nombres actuales de las columnas a los nuevos nombres.

        Returns:
            pd.DataFrame: El dataframe con los nombres de columnas modificados.
        """
        dataframe.rename(columns=column_mapping, inplace=True)
        return dataframe

    def get_duplicate_rows(self, dataframe, column):
        """
        Devuelve las filas del dataframe que tienen valores repetidos en una columna específica,
        junto con la cantidad de repeticiones y el porcentaje respecto al total de líneas.

        Args:
            dataframe (pd.DataFrame): El dataframe a analizar.
            column (str): El nombre de la columna en la cual buscar repeticiones.

        Returns:
            pd.DataFrame: Un dataframe con las filas que se repiten, la cantidad de repeticiones
                        y el porcentaje respecto al total de líneas.
        """
        duplicated_rows = dataframe[dataframe.duplicated(subset=column, keep=False)]
        duplicated_counts = duplicated_rows[column].value_counts().reset_index()
        duplicated_counts.columns = [column, 'Cantidad']
        duplicated_counts['Porcentaje'] = duplicated_counts['Cantidad'] / len(dataframe) * 100
        return duplicated_counts
    
    def get_duplicate_rows_2(self, dataframe, column):
        """
        Devuelve las filas del dataframe que tienen valores repetidos en una columna específica.
        Incluye las otras columnas del dataframe, la cantidad y el porcentaje de filas duplicadas.
        Ordena el resultado por los valores de la columna especificada.

        Args:
            dataframe (pd.DataFrame): El dataframe a analizar.
            column (str): El nombre de la columna en la que se buscarán valores duplicados.

        Returns:
            pd.DataFrame: El dataframe con las filas duplicadas, incluyendo las otras columnas,
                        la cantidad y el porcentaje de filas duplicadas. Ordenado por la columna especificada.
        """
        duplicates = dataframe[dataframe.duplicated(subset=column, keep=False)]
        duplicate_counts = duplicates[column].value_counts()
        duplicate_percentages = duplicate_counts / len(dataframe) * 100

        duplicates['Duplicate Count'] = duplicates[column].map(duplicate_counts)
        duplicates['Duplicate Percentage'] = duplicates[column].map(duplicate_percentages)

        duplicates = duplicates.sort_values(by=column)

        return duplicates

    def strip_column_values(self, dataframe, column_names):
        """
        Elimina los espacios en blanco al inicio y al final de los valores de las filas de las columnas especificadas.

        Args:
            dataframe (pd.DataFrame): El dataframe en el cual se van a limpiar los valores de las columnas.
            column_names (list): Una lista de nombres de columnas a las cuales se les va a aplicar el método .strip().

        Returns:
            pd.DataFrame: El dataframe con los valores de las filas de las columnas especificadas limpios de espacios en blanco.
        """
        for column in column_names:
            dataframe[column] = dataframe[column].str.strip()
        return dataframe

    def sort_dataframe(self, dataframe, column, order='a'):
        """
        Ordena un dataframe por una columna especificada.

        Args:
            dataframe (pd.DataFrame): El dataframe a ordenar.
            column (str): El nombre de la columna por la cual ordenar el dataframe.
            order (str, opcional): El orden de clasificación. 'a' para ascendente (predeterminado),
                'd' para descendente.

        Returns:
            pd.DataFrame: El dataframe ordenado por la columna especificada.
        """
        if order == 'd':
            dataframe = dataframe.sort_values(column, ascending=False)
        else:
            dataframe = dataframe.sort_values(column)

        return dataframe

    def sort_and_replace(self, dataframe, column1, column2):
        """
        Ordena un dataframe de menor a mayor según los valores de una columna y reemplaza los valores de otra columna por enteros.

        Args:
            dataframe (pd.DataFrame): El dataframe a procesar.
            column1 (str): El nombre de la primera columna según la cual se va a ordenar el dataframe.
            column2 (str): El nombre de la segunda columna cuyos valores se reemplazarán por enteros.

        Returns:
            pd.DataFrame: El dataframe ordenado y con los valores de la segunda columna reemplazados por enteros.
        """
        # Ordenar dataframe por la columna 1 de menor a mayor
        dataframe = dataframe.sort_values(by=column1)

        # Reemplazar los valores de la columna 2 por enteros
        dataframe[column2] = range(len(dataframe))

        return dataframe
        
    def convert_date_format(self, dataframe, columns):
        """
        Convierte el formato de fecha de columnas en un dataframe de 'MMM DD, YYYY' a 'YYYY-MM-DD'
        y asigna el dtype datetime a las columnas especificadas.

        Args:
            dataframe (pd.DataFrame): El dataframe a procesar.
            columns (list): Una lista de nombres de columnas a convertir y asignar el dtype datetime.

        Returns:
            pd.DataFrame: El dataframe con los formatos de fecha modificados y dtype datetime asignado.
        """
        for column in columns:
            dataframe[column] = pd.to_datetime(dataframe[column], format='%b %d, %Y').dt.strftime('%Y-%m-%d')
            dataframe[column] = pd.to_datetime(dataframe[column])
            
        return dataframe

    def remove_prefix(self, dataframe, column, prefix):
        """
        Elimina un prefijo de las filas de una columna en un dataframe.

        Args:
            dataframe (pd.DataFrame): El dataframe a procesar.
            column (str): El nombre de la columna en la que se eliminará el prefijo.
            prefix (str): El prefijo a eliminar de las filas de la columna.

        Returns:
            pd.DataFrame: El dataframe con el prefijo eliminado de las filas de la columna especificada.
        """
        dataframe[column] = dataframe[column].str.lstrip(prefix)
        return dataframe
    
    def transform_values(self, dataframe, columns, symbols):
        """
        Transforma los valores de las columnas especificadas en el dataframe, eliminando símbolos, realizando un strip y convirtiendo a minúscula.

        Args:
            dataframe (pd.DataFrame): El dataframe a transformar.
            columns (list): La lista de nombres de columnas a transformar.
            symbols (list): La lista de símbolos a eliminar.

        Returns:
            pd.DataFrame: El dataframe con los valores transformados.
        """
        for column in columns:
            for symbol in symbols:
                dataframe[column] = dataframe[column].str.replace(symbol, '')
            dataframe[column] = dataframe[column].str.strip().str.lower()

        return dataframe

    def replace_na_values_obj(self, dataframe, columns, replacement):
        """
        Reemplaza los valores NaN en las columnas especificadas del dataframe con un valor de reemplazo.

        Args:
            dataframe (pd.DataFrame): El dataframe en el que se reemplazarán los valores NaN.
            columns (list): La lista de nombres de columnas en las que se realizará el reemplazo.
            replacement (str): El valor de reemplazo para los valores NaN.

        Returns:
            pd.DataFrame: El dataframe con los valores NaN reemplazados.
        """
        dataframe[columns] = dataframe[columns].fillna(replacement)
        return dataframe

    def convert_values_to_int(self,dataframe, columns, str_to_remove):
        """
        Convierte los valores de las columnas seleccionadas del dataframe en enteros.

        Los valores de las columnas seleccionadas que contengan el str indicado se eliminan.
        Los valores restantes se convierten a enteros.

        Args:
            dataframe (pd.DataFrame): El dataframe original.
            columns (list): Lista de nombres de columnas a modificar.
            str_to_remove (str): El str a eliminar de los valores de las columnas.

        Returns:
            pd.DataFrame: El dataframe modificado con los valores convertidos a enteros.
        """
        for column in columns:
            dataframe[column] = dataframe[column].str.replace(str_to_remove, '')
            dataframe[column] = pd.to_numeric(dataframe[column], errors='coerce').astype(pd.Int64Dtype())

        return dataframe

# Instancia de la clase EDA
eda = EDA()

<font size="4"><li><left><i>Exportamos udemy_courses.csv a dataframe</i></left></font>

In [2]:
# Path de archivos CSV
path = '/Users/negro/Library/CloudStorage/OneDrive-Personal/Documentos/00 Fran/01 - Personales/02-Learn/0. Data Science/0. Data Science/2_projects/e_MOOCs/2_Repo/MOOCs_BA/rawDataSets/udemy_courses.csv'

# Leer y almacenar los archivos CSV en dataframes
udemy_courses = eda.read_csv(path)

<font size="4"><li><left><i>Obtenemos información del dataframe</i></left></font>

In [3]:
udemy_courses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3678 entries, 0 to 3677
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   course_id            3678 non-null   int64  
 1   course_title         3678 non-null   object 
 2   url                  3678 non-null   object 
 3   is_paid              3678 non-null   bool   
 4   price                3678 non-null   int64  
 5   num_subscribers      3678 non-null   int64  
 6   num_reviews          3678 non-null   int64  
 7   num_lectures         3678 non-null   int64  
 8   level                3678 non-null   object 
 9   content_duration     3678 non-null   float64
 10  published_timestamp  3678 non-null   object 
 11  subject              3678 non-null   object 
dtypes: bool(1), float64(1), int64(5), object(5)
memory usage: 319.8+ KB


In [ ]:
# Generar el perfil del dataframe
profile = udemy_courses.profile_report(title='Informe de Pandas-Profiling', explorative=True)
# Guardar el informe en un archivo HTML
profile.to_file('/Users/negro/Library/CloudStorage/OneDrive-Personal/Documentos/00 Fran/01 - Personales/02-Learn/0. Data Science/0. Data Science/2_projects/e_MOOCs/2_Repo/MOOCs_BA/reports/udemy_courses.html')
# Cargar y mostrar el informe HTML
report_html = open('/Users/negro/Library/CloudStorage/OneDrive-Personal/Documentos/00 Fran/01 - Personales/02-Learn/0. Data Science/0. Data Science/2_projects/e_MOOCs/2_Repo/MOOCs_BA/reports/udemy_courses.html', 'r').read()
display(HTML(report_html))


In [6]:
eda.get_duplicate_rows(udemy_courses, 'course_id')

,course_id,Cantidad,Porcentaje
0,837322,2,0.054377
1,1157298,2,0.054377
2,1084454,2,0.054377
3,1035638,2,0.054377
4,185526,2,0.054377
5,28295,2,0.054377


In [16]:
eda.get_duplicate_rows_2(udemy_courses, 'course_id').head(50)

,course_id,course_title,url,is_paid,price,num_subscribers,num_reviews,num_lectures,level,content_duration,published_timestamp,subject,Duplicate Count,Duplicate Percentage
2478,28295,Learn Web Designing & HTML5/CSS3 Essentials in...,https://www.udemy.com/build-beautiful-html5-we...,True,75,43285,525,24,All Levels,4.000000,2013-01-03T00:55:31Z,Web Development,2,0.054377
2561,28295,Learn Web Designing & HTML5/CSS3 Essentials in...,https://www.udemy.com/build-beautiful-html5-we...,True,75,43285,525,24,All Levels,4.000000,2013-01-03T00:55:31Z,Web Development,2,0.054377
1234,185526,MicroStation - Células,https://www.udemy.com/microstation-celulas/,True,20,0,0,9,Beginner Level,0.616667,2014-04-15T21:48:55Z,Graphic Design,2,0.054377
1473,185526,MicroStation - Células,https://www.udemy.com/microstation-celulas/,True,20,0,0,9,Beginner Level,0.616667,2014-04-15T21:48:55Z,Graphic Design,2,0.054377
453,837322,Essentials of money value: Get a financial Life !,https://www.udemy.com/essentials-of-money-value/,True,20,0,0,20,All Levels,0.616667,2016-05-16T18:28:30Z,Business Finance,2,0.054377
787,837322,Essentials of money value: Get a financial Life !,https://www.udemy.com/essentials-of-money-value/,True,20,0,0,20,All Levels,0.616667,2016-05-16T18:28:30Z,Business Finance,2,0.054377
778,1035638,Understanding Financial Statements,https://www.udemy.com/understanding-financial-...,True,25,0,0,10,All Levels,1.000000,2016-12-15T14:56:17Z,Business Finance,2,0.054377
894,1035638,Understanding Financial Statements,https://www.udemy.com/understanding-financial-...,True,25,0,0,10,All Levels,1.000000,2016-12-15T14:56:17Z,Business Finance,2,0.054377
463,1084454,CFA Level 2- Quantitative Methods,https://www.udemy.com/cfa-level-2-quantitative...,True,40,0,0,35,All Levels,5.500000,2017-07-02T14:29:35Z,Business Finance,2,0.054377
1100,1084454,CFA Level 2- Quantitative Methods,https://www.udemy.com/cfa-level-2-quantitative...,True,40,0,0,35,All Levels,5.500000,2017-07-02T14:29:35Z,Business Finance,2,0.054377


<font size="4"><li><left><i>Se identifican valores null en este dataset en columnas del tipo `object` e `int`, analizamos la cabezera:</i></left></font><br>

In [43]:
# Obtener cabecera del dataframe
edx_courses.head()

,title,summary,n_enrolled,course_type,institution,instructors,Level,subject,language,subtitles,course_effort,course_length,price,course_description,course_syllabus,course_url
0,How to Learn Online,Learn essential strategies for successful onli...,"124,980",Self-paced on your time,edX,Nina Huntemann-Robyn Belair-Ben Piscopo,Introductory,Education & Teacher Training,English,English,2–3 hours per week,2 Weeks,FREE-Add a Verified Certificate for $49 USD,"Designed for those who are new to elearning, t...",Welcome - We start with opportunities to meet ...,https://www.edx.org/course/how-to-learn-online
1,Programming for Everybody (Getting Started wit...,"This course is a ""no prerequisite"" introductio...","293,864",Self-paced on your time,The University of Michigan,Charles Severance,Introductory,Computer Science,English,English,2–4 hours per week,7 Weeks,FREE-Add a Verified Certificate for $49 USD,This course aims to teach everyone the basics ...,NaN,https://www.edx.org/course/programming-for-eve...
2,CS50's Introduction to Computer Science,An introduction to the intellectual enterprise...,"2,442,271",Self-paced on your time,Harvard University,David J. Malan-Doug Lloyd-Brian Yu,Introductory,Computer Science,English,English,6–18 hours per week,12 Weeks,FREE-Add a Verified Certificate for $90 USD,"This is CS50x , Harvard University's introduct...",NaN,https://www.edx.org/course/cs50s-introduction-...
3,The Analytics Edge,"Through inspiring examples and stories, discov...","129,555",Instructor-led on a course schedule,Massachusetts Institute of Technology,Dimitris Bertsimas-Allison O'Hair-John Silberh...,Intermediate,Data Analysis & Statistics,English,English,10–15 hours per week,13 Weeks,FREE-Add a Verified Certificate for $199 USD,"In the last decade, the amount of data availab...",NaN,https://www.edx.org/course/the-analytics-edge
4,Marketing Analytics: Marketing Measurement Str...,This course is part of a MicroMasters® Program,"81,140",Self-paced on your time,"University of California, Berkeley",Stephan Sorger,Introductory,Computer Science,English,English,5–7 hours per week,4 Weeks,FREE-Add a Verified Certificate for $249 USD,Begin your journey in a new career in marketin...,NaN,https://www.edx.org/course/marketing-analytics...


<font size="4"><li><left><i>Asignamos tipos de datos:</i></left></font>

In [38]:
# Queremos reemplazar los valores NaN en las columnas de 'columns_to_fill' con el valor 'N/A' dado que son considerados atributos del tipo object
columns_to_convert = ['summary', 'instructors','course_description']
edx_courses = eda.set_columns_dtype_category(edx_courses, columns_to_convert)

# Queremos eliminar el str ',' de las columnas seleccionadas y convertir los valores a enteros.
columns_to_convert = ['n_enrolled']
edx_courses = eda.convert_values_to_int(edx_courses, columns_to_convert, ',')

# Obtener cabecera del dataframe
edx_courses.head()

TypeError: 'NoneType' object is not subscriptable

<font size="4"><li><left><i>Clasificamos variables y renombramos columnas:</i></left></font>

In [14]:
# Obtener lista de columnas
columnas = eda.get_columnas(edx_courses)
print(columnas)

['title', 'summary', 'n_enrolled', 'course_type', 'institution', 'instructors', 'Level', 'subject', 'language', 'subtitles', 'course_effort', 'course_length', 'price', 'course_description', 'course_syllabus', 'course_url']


, , 'course_description', 'course_syllabus', 'course_url'

<font size="4"><li><left><i>Definimos variables cualitativas:</i></left></font><br>
<br>
<font size="4"><left><i>- Ordinales: `Level`</i></left></font><br>
<br>
<font size="4"><left><i>- Nominales: `title`, `summary`, `course_type`, `institution`, `instructors`,`subject`, `language`, `subtitles`,`course_description`, `course_syllabus` y `course_url`</i></left></font><br>
<br>
<font size="4"><li><left><i>Definimos variables cuantitativas:</i></left></font><br>
<br>
<font size="4"><left><i>- Discretas: `n_enrolled`, `course_effort` y `course_length` </i></left></font><br>
<br>
<font size="4"><left><i>- Continuas: `price` </i></left></font><br>
<br>
<br><font size="4"><li><left><i>Posible PK:</i></left></font><br>
<br>
<font size="4"><left><i>- `course_id`</i></left></font><br>


In [23]:
# Parametrización de asiganción de Dtypes:
# Asignar tipo de dato 'category'
columnas_cat = ['title', 'course_type', 'institution', 'Level', 'subject', 'language', 'subtitles', 'course_effort']
edx_courses = eda.set_columns_dtype_category(edx_courses,columnas_cat)

# Asignar tipo de dato 'int' a las columnas
columnas_int = ['n_enrolled']
edx_courses = eda.set_columns_dtype_int(edx_courses, columnas_int)

# Asignar tipo de dato 'float' a las columnas
# columnas_float = []
# eda.set_columns_dtype_float(dataframe, columnas_float)

# Asignar tipo de dato 'datetime' a la columna con formato 'YYYY-MM-DD'
# columna_fecha = []
# eda.set_columns_dtype_datetime(dataframe, columna_fecha, formato='%Y-%m-%d')

# Crear una lista con las columnas que objetos con formato daytime, asiganar en formato 'MMM DD, YYYY'
# columnas_fecha = ['date_reviews']
# Coursera_reviews = eda.convert_date_format(Coursera_reviews, columnas_fecha)


TypeError: 'NoneType' object is not subscriptable

In [124]:
# Obtener cabecera del dataframe
Coursera_reviews.head()

,reviews,reviewers,date_reviews,rating,course_id
0,"Pretty dry, but I was able to pass with just t...",By Robert S,2020-02-12,4,google-cbrs-cpi-training
1,would be a better experience if the video and ...,By Gabriel E R,2020-09-28,4,google-cbrs-cpi-training
2,Information was perfect! The program itself wa...,By Jacob D,2020-04-08,4,google-cbrs-cpi-training
3,A few grammatical mistakes on test made me do ...,By Dale B,2020-02-24,4,google-cbrs-cpi-training
4,Excellent course and the training provided was...,By Sean G,2020-06-18,4,google-cbrs-cpi-training


In [125]:
Coursera_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1454711 entries, 0 to 1454710
Data columns (total 5 columns):
 #   Column        Non-Null Count    Dtype         
---  ------        --------------    -----         
 0   reviews       1454571 non-null  object        
 1   reviewers     1454711 non-null  object        
 2   date_reviews  1454711 non-null  datetime64[ns]
 3   rating        1454711 non-null  int64         
 4   course_id     1454711 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 55.5+ MB


<font size="4"><li><left><i>Renombramos columnas:</i></left></font>

In [126]:
# Definir un diccionario de mapeo de nombres de columnas
column_mapping = {'reviews':'resenia', 'reviewers': 'nombre_revisor', 'date_reviews':'fecha_resenia', 'rating':'calificacion', 'course_id':'id_curso'}

# Llamar al método rename_columns
dataframe_modificado = eda.rename_columns(Coursera_reviews, column_mapping)


In [107]:
Coursera_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1454711 entries, 0 to 1454710
Data columns (total 5 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   resenia         1454571 non-null  object        
 1   nombre_revisor  1454711 non-null  object        
 2   fecha_resenia   1454711 non-null  datetime64[ns]
 3   calificacion    1454711 non-null  int64         
 4   id_curso        1454711 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 55.5+ MB


<font size="4" color="white"><li><left><i>Eliminamos espacios antes/despues:</i></left></font>

In [127]:
# Eliminar espacios alrededor de str
# Definir una lista de nombres de columnas a limpiar
columnas_a_limpiar = ['resenia', 'nombre_revisor','id_curso']

# Llamar al método strip_column_values
Coursera_reviews = eda.strip_column_values(Coursera_reviews, columnas_a_limpiar)

<font size="4" color="white"><li><left><i>Eliminamos el prefijo `By `de los valores de la columna `nombre_revisor`:</i></left></font>

In [128]:
Coursera_reviews = eda.remove_prefix(Coursera_reviews, 'nombre_revisor', 'By ')
Coursera_reviews.head()

,resenia,nombre_revisor,fecha_resenia,calificacion,id_curso
0,"Pretty dry, but I was able to pass with just t...",Robert S,2020-02-12,4,google-cbrs-cpi-training
1,would be a better experience if the video and ...,Gabriel E R,2020-09-28,4,google-cbrs-cpi-training
2,Information was perfect! The program itself wa...,Jacob D,2020-04-08,4,google-cbrs-cpi-training
3,A few grammatical mistakes on test made me do ...,Dale B,2020-02-24,4,google-cbrs-cpi-training
4,Excellent course and the training provided was...,Sean G,2020-06-18,4,google-cbrs-cpi-training


<font size="4" color="white"><li><left><i>Ordenamos por columna:</i></left></font>

In [129]:
# Llamar al método sort_dataframe
Coursera_reviews = eda.sort_dataframe(Coursera_reviews, 'calificacion','d')
Coursera_reviews.head()

,resenia,nombre_revisor,fecha_resenia,calificacion,id_curso
1454710,G,Reza S,2018-08-12,5,computer-networking
783268,I love it. Learnt a lot from this online course.,Tan Y S,2020-05-12,5,search-engine-optimization
783274,One of my best course that I want to finish it .,Fadli I,2016-02-10,5,search-engine-optimization
783273,Thank you so much for providing me this platform,Simranjit S,2016-10-14,5,search-engine-optimization
783272,A very useful introduction to the basics of SEO.,Alejandro F y B,2019-09-25,5,search-engine-optimization


 <font size="4" color="white"><li><left><i>Analizamos duplicados:</i></left></font>

In [132]:
# Traer filas con duplicados en 'resenia'
filas_duplicadas = eda.get_duplicate_rows(Coursera_reviews, 'resenia')
filas_duplicadas.head(10)

,resenia,Cantidad,Porcentaje
0,good,9236,0.634903
1,Good,6106,0.419740
2,Excellent,4656,0.320064
3,Great course!,2796,0.192203
4,Excellent course,2409,0.165600
5,great,2377,0.163400
6,Great course,2350,0.161544
7,excellent,2313,0.159001
8,very good,2279,0.156663
9,nice,2245,0.154326


<font size="4"><li><left><i>La reseña que más se repite es `good`, pero se nota que hay diferencias en valores que consideraremos iguales.</i></left></font>
<br>
<br>
<font size="4"><li><left><i>Armonizamos la fila para agrupar de mejor manera:</i></left></font>

In [135]:
# Reemplzar caracteres irrelevantes y homogenizar a minúsculas
columns_to_transform = ['resenia']
symbols_to_remove = ['!', 'course', 'curso', 'Curso', 'Course']

Coursera_reviews = eda.transform_values(Coursera_reviews, columns_to_transform, symbols_to_remove)

# Traer filas con duplicados en 'resenia'
filas_duplicadas = eda.get_duplicate_rows(Coursera_reviews, 'resenia')
filas_duplicadas.head(10)

,resenia,Cantidad,Porcentaje
0,good,21349,1.467577
1,excellent,17860,1.227735
2,great,17421,1.197557
3,very good,7504,0.515841
4,nice,6789,0.466691
5,excelente,6439,0.442631
6,awesome,5450,0.374645
7,amazing,4011,0.275725
8,very useful,2204,0.151508
9,thank you,2136,0.146833


<font size="4"><li><left><i>La distribución de `resenia` muestra una alta disperción, con algunos valores relevantes como `good`, `excellent` y `great`.</i></left></font>

In [136]:
# Traer filas con duplicados en 'nombre_revisor'
filas_duplicadas = eda.get_duplicate_rows(Coursera_reviews, 'nombre_revisor')
filas_duplicadas.head(10)

,nombre_revisor,Cantidad,Porcentaje
0,Deleted A,5412,0.372033
1,Muhammad A,664,0.045645
2,Abhishek S,440,0.030247
3,David M,439,0.030178
4,Michael S,431,0.029628
5,Michael M,400,0.027497
6,David S,381,0.026191
7,Michael B,380,0.026122
8,David B,378,0.025985
9,Aditya S,377,0.025916


<font size="4"><li><left><i>El `nombre_revisor` que más se repite es `Deleted A` y representa un `0,37%` del total, esperada alta disperción en este atributo.  </i></left></font>

In [137]:
# Traer filas con duplicados en 'fecha_resenia'
filas_duplicadas = eda.get_duplicate_rows(Coursera_reviews, 'fecha_resenia')
filas_duplicadas.head(5)


,fecha_resenia,Cantidad,Porcentaje
0,2020-05-25,5394,0.370795
1,2020-05-18,5391,0.370589
2,2020-06-01,5373,0.369352
3,2020-05-31,5359,0.368389
4,2020-05-11,5310,0.365021


<font size="4"><li><left><i>La distribución de `fecha_resenia` muestra una alta disperción.</i></left></font>

In [138]:
# Traer filas con duplicados en 'calificacion'
filas_duplicadas = eda.get_duplicate_rows(Coursera_reviews, 'calificacion')
filas_duplicadas.head(20)

,calificacion,Cantidad,Porcentaje
0,5,1146164,78.789808
1,4,226702,15.583989
2,3,48303,3.320453
3,1,17354,1.192952
4,2,16188,1.112798


<font size="4"><li><left><i>Más de un `78%` de `calificacion` es igual a `5`.</i></left></font>

In [139]:
# Traer filas con duplicados en 'id_curso'
filas_duplicadas = eda.get_duplicate_rows(Coursera_reviews, 'id_curso')
filas_duplicadas.head(20)

,id_curso,Cantidad,Porcentaje
0,python,45218,3.108384
1,machine-learning,35895,2.467500
2,neural-networks-deep-learning,35750,2.457533
3,technical-support-fundamentals,35319,2.427905
4,python-data,33546,2.306025
5,learning-how-to-learn,32265,2.217966
6,ai-for-everyone,21630,1.486893
7,excel-essentials,21251,1.460840
8,what-is-datascience,21087,1.449566
9,the-science-of-well-being,20085,1.380687


<font size="4"><li><left><i>El `id_curso` con más `resenia` es `python` con `78%` de `45.218`, esto es un `25%` más que el inmediato segundo `machine-learning`.</i></left></font>